In [28]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import talib as ta

In [29]:
df = pd.read_csv('../../../data/stocks/AAPL.csv')
temp = df.copy()
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


# Buying and Selling over SMA intersection Strategy

## Strategy Overview
This is a **trend-following strategy** that uses a single Simple Moving Average (SMA) as a primary signal for entry and exit points. The fundamental idea is to buy when the price is above the moving average (indicating an uptrend) and sell when the price falls below it (indicating a downtrend).

## Core Concept & Theory

### What is SMA (Simple Moving Average)?
The SMA is the arithmetic mean of the last N periods of price data. It smooths out price volatility to reveal underlying trend direction. A 50-period SMA is commonly used for medium-term trend identification.

**Formula:**
$$SMA_{50} = \frac{P_1 + P_2 + ... + P_{50}}{50}$$

Where P represents closing prices.

### Signal Generation Logic
- **BUY Signal**: When the current close price crosses **above** the 50-period SMA
  - Interpretation: Price momentum is turning positive; bullish sentiment emerging
  - Market Psychology: Buyers are becoming more aggressive than sellers
  
- **SELL Signal**: When the current close price crosses **below** the 50-period SMA
  - Interpretation: Price momentum is turning negative; bearish sentiment emerging
  - Market Psychology: Sellers are gaining control

### Why This Works
1. **Trend Confirmation**: SMA filters out noise and short-term price fluctuations
2. **Dynamic Support/Resistance**: The MA acts as a dynamic level around which price oscillates
3. **Momentum Validation**: Crossing the MA indicates a shift in buying/selling pressure
4. **Simple & Objective**: Clear, rule-based entry/exit criteria remove emotional decision-making

### Risk Considerations
- **Whipsaws in Sideways Markets**: In ranging/choppy markets, the strategy generates false signals
- **Lag**: Moving averages are lagging indicators—signals occur after the move has begun
- **Position Sizing**: Without proper risk management, winning trades don't outweigh losses from false signals

In [30]:
def buy_sell_over_SMA_intersection_strategy(df: pd.DataFrame) -> pd.DataFrame:
    """
    Implements a simple SMA crossover strategy with a single 50-period moving average.
    
    Args:
        df: DataFrame with 'Close' prices
        
    Returns:
        DataFrame with added SMA_50, Signal, and Entry/Exit columns
        
    Logic Flow:
    1. Calculate 50-period SMA - smoothed trend indicator
    2. Generate signal: 1 if Close > SMA (bullish), 0 otherwise (bearish)
    3. Detect crossovers using diff() - captures points where signal changes
    """
    # Calculate 50-period Simple Moving Average
    # This value serves as the "fair value" or trend baseline
    df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
    
    # Generate trading signal based on price position relative to SMA
    # Signal = 1: Price above SMA (bullish regime) → favorable for buying
    # Signal = 0: Price below SMA (bearish regime) → favorable for selling
    df['Signal'] = np.where(df['Close'] > df['SMA_50'], 1, 0)
    
    # Calculate Entry/Exit points by detecting signal transitions
    # Entry/Exit = 1: Golden Cross (Signal changes from 0 to 1) → BUY signal
    # Entry/Exit = -1: Death Cross (Signal changes from 1 to 0) → SELL signal
    # Entry/Exit = 0: No change in regime → Hold position
    df['Entry/Exit'] = df['Signal'].diff()
    
    return df

In [31]:
def plot_buy_sell_over_SMA_intersection_strategy(df: pd.DataFrame) -> go.Figure:
    fig = go.Figure(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )
    )
    fig.add_trace(
        go.Scatter(
            name='50_SMA',
            x=df['Date'],
            y=df['SMA_50'],
            line=dict(color='blue', width=1),
        )
    )

    # Filter data for Buy and Sell points
    buys = df[df['Entry/Exit'] == 1]
    sells = df[df['Entry/Exit'] == -1]

    # Add Buy Markers (Green Triangles)
    fig.add_trace(
        go.Scatter(
            x=buys['Date'],
            y=buys['Low'] * 0.99,  # Offset slightly below the candle
            mode='markers',
            name='Buy Signal',
            marker=dict(
                symbol='triangle-up',
                size=12,
                color='green',
            ),
        )
    )

    # Add Sell Markers (Red Triangles)
    fig.add_trace(
        go.Scatter(
            x=sells['Date'],
            y=sells['High'] * 1.01,  # Offset slightly above the candle
            mode='markers',
            name='Sell Signal',
            marker=dict(
                symbol='triangle-down',
                size=12,
                color='red',
            ),
        )
    )

    fig.update_layout(
        title='AAPL SMA 50 Strategy 1 - Buy/Sell on SMA Intersection',
        xaxis_rangeslider_visible=False,
        yaxis_title='Price (USD)',
        template='plotly_dark' 
    )

    return fig

In [32]:
df = buy_sell_over_SMA_intersection_strategy(df)
plot_buy_sell_over_SMA_intersection_strategy_figure = plot_buy_sell_over_SMA_intersection_strategy(df)
plot_buy_sell_over_SMA_intersection_strategy_figure.show()

# SMA Crossover Strategy (Golden Cross & Death Cross)

## Strategy Overview
This is an advanced **trend-following strategy** that uses **two SMAs of different periods** to identify major trend reversals. It's based on the concept of "Golden Cross" (bullish) and "Death Cross" (bearish) signals—classic technical analysis patterns used by institutional traders.

## Core Concept & Theory

### What is a Moving Average Crossover?
When a faster-moving average (shorter period, more responsive) crosses above a slower-moving average (longer period, more stable), it signals a bullish shift in momentum. Conversely, when the faster MA crosses below the slower MA, it signals a bearish shift.

### The Golden Cross (Buy Signal)
- **Occurs When**: 50-day SMA > 200-day SMA (faster MA crosses above slower MA)
- **Interpretation**: 
  - Short-term momentum is accelerating upward
  - Medium-term uptrend (50-period) is overpowering long-term trend (200-period)
  - Institutional buyers are stepping in; bullish sentiment confirmed
- **Historical Significance**: Golden Crosses on major indices have historically preceded significant rallies

### The Death Cross (Sell Signal)
- **Occurs When**: 50-day SMA < 200-day SMA (faster MA crosses below slower MA)
- **Interpretation**:
  - Short-term momentum is decelerating downward
  - Medium-term downtrend (50-period) is breaking below long-term support (200-period)
  - Institutional sellers are stepping in; bearish sentiment confirmed
- **Historical Significance**: Death Crosses have often preceded market corrections or bear markets

## Why 50 and 200 Periods?

### 50-Period SMA (Medium-Term Trend)
- Represents approximately 2-3 months of trading data (50 trading days)
- Responsive enough to catch trend changes without excessive noise
- Widely used institutional standard for medium-term analysis
- Acts as dynamic support/resistance

### 200-Period SMA (Long-Term Trend)
- Represents approximately 9-10 months of trading data (200 trading days)
- Establishes the longer-term directional bias
- Less responsive to short-term volatility; reflects structural market trends
- Gold standard in technical analysis; used by hedge funds, pension funds, etc.
- Often called the "200-day line" — a critical level in financial markets

### The Power of the 50/200 Combination
The crossover of these two SMAs is **more significant** than a single MA because:
1. **Double Confirmation**: Requires both short AND long-term momentum shifts
2. **Filter Effect**: Reduces false signals from random price noise
3. **Institutional Adoption**: Many institutional traders use this exact setup
4. **Self-Fulfilling Prophecy**: Because it's widely used, major moves often occur at these crossovers

## Mathematical Logic

**Buy Signal Generation:**
$$\text{Entry} = \begin{cases} 1 & \text{if } SMA_{50} > SMA_{200} \\ 0 & \text{otherwise} \end{cases}$$

**Sell Signal Occurs When:**
Entry signal changes from 1 to 0 (transition from uptrend to downtrend)

## Advantages Over Single MA Strategy
1. **Higher Quality Signals**: Filters out whipsaws in choppy markets
2. **Trend Alignment**: Only trades in direction of primary (200-day) trend
3. **Lower Transaction Costs**: Fewer false signals = fewer trades = lower fees
4. **Professional Standard**: Aligns with institutional trading practices
5. **Proven Track Record**: This strategy has documented success across multiple markets and decades

## Risk Considerations
1. **Lag in Downtrends**: By the time Death Cross occurs, significant losses may already be realized
2. **Fast Market Moves**: If gaps occur, execution at MA price may not be possible
3. **Choppy Markets**: Extended periods of sideways action can still generate multiple crossovers
4. **No Stop Loss Built-in**: Strategy needs position sizing and exit rules to manage drawdowns
5. **Data Requirements**: Requires 200 periods of historical data before generating any signals

In [33]:
df = temp
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


In [34]:
def SMA_crossover_strategy(df: pd.DataFrame) -> pd.DataFrame:
    """
    Implements the Golden Cross / Death Cross strategy using 50 and 200-period SMAs.
    
    This is one of the most widely used institutional trading strategies.
    
    Args:
        df: DataFrame with 'Close' prices
        
    Returns:
        DataFrame with added SMA_50, SMA_200, Signal, and Entry/Exit columns
        
    Trading Logic:
    1. Calculate two SMAs representing different timeframes
    2. Compare them to determine overall trend health
    3. Generate signals when the relationship between them changes
    4. Entry/Exit captures the exact moment of crossover (high-confidence signals)
    """
    
    # Calculate 50-period SMA (medium-term trend indicator)
    # Responds to recent price action; captures intermediate momentum shifts
    # TA-Lib's SMA uses the standard formula: simple average of last 50 closes
    df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
    
    # Calculate 200-period SMA (long-term trend indicator)
    # Represents the macro trend; filters out intermediate noise
    # Only responds to sustained price movements; very stable baseline
    df['SMA_200'] = ta.SMA(df['Close'], timeperiod=200)
    
    # Generate signal based on relative MA positions
    # Signal = 1: SMA_50 > SMA_200 (Golden Cross regime - BULLISH)
    #   → Price trend is accelerating upward
    #   → Medium-term momentum dominates; we're in an uptrend
    #   → Favorable for long positions
    #
    # Signal = 0: SMA_50 < SMA_200 (Death Cross regime - BEARISH)
    #   → Price trend is decelerating/reversing; downtrend
    #   → Long-term momentum dominates; caution advised
    #   → Favorable for avoiding losses or short positions
    df['Signal'] = np.where(df['SMA_50'] > df['SMA_200'], 1, 0)
    
    # Detect actual crossover points (Entry/Exit signals)
    # Entry/Exit = 1: Golden Cross (Signal changes from 0→1)
    #   → SMA_50 just crossed ABOVE SMA_200
    #   → High-conviction BUY signal from institutional traders
    #   → Market is transitioning to a bullish bias
    #
    # Entry/Exit = -1: Death Cross (Signal changes from 1→0)
    #   → SMA_50 just crossed BELOW SMA_200
    #   → High-conviction SELL signal from institutional traders
    #   → Market is transitioning to a bearish bias
    #
    # Entry/Exit = 0: No crossover occurring
    #   → We're in a stable trend (either bullish or bearish)
    #   → No new signal; maintain current position
    df['Entry/Exit'] = df['Signal'].diff()
    
    return df

In [35]:
def plot_SMA_crossover_strategy(df: pd.DataFrame) -> go.Figure:
    fig = go.Figure(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Market Price'
        )
    )
    
    # Add SMA Traces
    fig.add_trace(
        go.Scatter(
            x=df['Date'], 
            y=df['SMA_50'], 
            name='50 SMA', 
            line=dict(color='blue', width=1.5)
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'], 
            y=df['SMA_200'], 
            name='200 SMA', 
            line=dict(color='red', width=1.5)
        )
    )

    # Buy and Sell signal filtering
    buys = df[df['Entry/Exit'] == 1]
    sells = df[df['Entry/Exit'] == -1]

    # Add Markers
    fig.add_trace(go.Scatter(
        x=buys['Date'], 
        y=buys['Low'] * 0.97, 
        mode='markers', 
        name='Golden Cross (Buy)', 
        marker=dict(symbol='triangle-up', size=13, color='#00ff00')
    ))
    fig.add_trace(go.Scatter(
        x=sells['Date'], 
        y=sells['High'] * 1.03, 
        mode='markers', 
        name='Death Cross (Sell)', 
        marker=dict(symbol='triangle-down', size=13, color='#ff0000')
    ))

    fig.update_layout(
        title='AAPL Golden Cross Strategy (50 vs 200 SMA)',
        xaxis_rangeslider_visible=False,
        yaxis_title='Price (USD)',
        template='plotly_dark' 
    )

    return fig

In [36]:
df = SMA_crossover_strategy(df)
plot_SMA_crossover_strategy_figure = plot_SMA_crossover_strategy(df)
plot_SMA_crossover_strategy_figure.show()